In [2]:
import pandas as pd
from ta.trend import MACD
from backtesting import Backtest, Strategy
import statsmodels.api as sm

Loading BokehJS ...

In [3]:
full_data = pd.read_csv('SPY_2024_15min.CSV', index_col=0)
full_data['date'] = pd.to_datetime(full_data['date'])

In [4]:
indicator_mcad = MACD(close=full_data['Close'])
full_data['macd'] = indicator_mcad.macd()
full_data['macd_signal'] = indicator_mcad.macd_signal()
full_data 

,date,Open,Low,High,Close,Volume,macd,macd_signal
6625,2021-01-04 09:30:00,375.310,373.54,375.4500,374.110,3829335,NaN,NaN
6626,2021-01-04 09:45:00,374.100,373.11,374.3400,373.510,2475997,NaN,NaN
6627,2021-01-04 10:00:00,373.530,372.56,373.7400,372.570,2135845,NaN,NaN
6628,2021-01-04 10:15:00,372.575,372.14,372.7800,372.490,2721962,NaN,NaN
6629,2021-01-04 10:30:00,372.480,371.70,372.5999,371.985,3569046,NaN,NaN
...,...,...,...,...,...,...,...,...
33509,2024-12-31 14:45:00,585.080,584.87,586.3400,586.330,1399535,-1.058092,-1.088467
33510,2024-12-31 15:00:00,586.310,585.17,586.8300,585.390,1728789,-1.038301,-1.078434
33511,2024-12-31 15:15:00,585.390,585.19,587.3000,587.230,1869350,-0.864182,-1.035583
33512,2024-12-31 15:30:00,587.230,586.03,587.5000,586.270,2116298,-0.794497,-0.987366


In [17]:
class LinRegStrat(Strategy):
    tp = 0.01
    sl = 0.01
    
    long_min_grad = 1
    short_max_grad = -1
    
    time_since_crossover = 1
    
    window = 500
    
    def init(self):
        ...
        
    def next(self):
        try:
            current_price = self.data.Close[-1]
            
            # MACD
            macd = self.data.macd[-self.time_since_crossover]
            macd_prev = self.data.macd[-(self.time_since_crossover + 1)]
            
            macd_signal = self.data.macd_signal[-self.time_since_crossover]
            macd_signal_prev = self.data.macd_signal[-(self.time_since_crossover + 1)]
            
            # Linear regression
            previous_prices = self.data.Close[-self.window:len(self.data.Close)]
            
            model = sm.OLS(previous_prices, [x for x in range(len(previous_prices))])
            results = model.fit()
            grad = results.params[0]
            
            if all([
                macd > macd_signal,
                macd_prev < macd_signal_prev,
                grad >= self.long_min_grad
            ]):
                self.buy(tp=current_price * (1 + self.tp), sl=current_price * (1 - self.sl))
                
            if all([
                macd < macd_signal,
                macd_prev > macd_signal_prev,
                grad <= self.short_max_grad
            ]):
                self.sell(tp=current_price * (1 - self.tp), sl=current_price * (1 + self.sl))
            
        except Exception as e:
            ...

In [18]:
bt = Backtest(full_data, LinRegStrat, cash=100_000, commission=.002, margin=1/5, exclusive_orders=False, hedging=True)
bt.run()

Start                                  6625.0
End                                   33513.0
Duration                              26888.0
Exposure Time [%]                   69.924504
Equity Final [$]                  3458.008244
Equity Peak [$]                 106770.497336
Return [%]                         -96.541992
Buy & Hold Return [%]               56.641095
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -97.159994
Avg. Drawdown [%]                  -19.105295
Max. Drawdown Duration                26498.0
Avg. Drawdown Duration                 4468.0
# Trades                                621.0
Win Rate [%]                        54.267311
Best Trade [%]                        3.25377
Worst Trade [%]                     -4.145154
Avg. Trade [%]                    

In [19]:
stats = bt.optimize(
    long_min_grad=[n / 100 for n in range(200)],
    short_max_grad=[-n / 100 for n in range(200)],
    tp=[0.0025, 0.005, 0.01, 0.025, 0.05],
    sl=[0.0025, 0.005, 0.01, 0.025, 0.05],
    time_since_crossover=[1, 2, 4, 8, 16, 32],
    window=[50, 100, 200, 300, 400, 500, 1000],
    constraint=lambda p: p.tp > p.sl,
    max_tries=200,
    random_state=5
)
stats

  0%|          | 0/9 [00:00<?, ?it/s]

Start                                  6625.0
End                                   33513.0
Duration                              26888.0
Exposure Time [%]                   98.787608
Equity Final [$]                204711.020324
Equity Peak [$]                 255001.027004
Return [%]                          104.71102
Buy & Hold Return [%]               56.641095
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -90.34256
Avg. Drawdown [%]                   -2.636855
Max. Drawdown Duration                19575.0
Avg. Drawdown Duration             154.052023
# Trades                                437.0
Win Rate [%]                        44.622426
Best Trade [%]                       7.209587
Worst Trade [%]                      -6.98773
Avg. Trade [%]                    